In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import time
import random
import datetime
from sklearn.metrics.pairwise import cosine_similarity
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split

In [2]:
cd /content

/content


In [4]:
df = pd.read_csv('query.csv')
df

,Query,Labels
0,What makes the ground shake during an earthquake?,Cause
1,What causes the earth to tremble and buildings...,Cause
2,What happens underground to create an earthquake?,Cause
3,What causes the earth to shake violently durin...,Cause
4,What are the factors that contribute to the oc...,Cause
...,...,...
305,Has there been any news on the environmental i...,News
306,Are there any reports on the economic conseque...,News
307,Has there been any news on technological advan...,News
308,Are there any reports on community initiatives...,News


In [5]:
possible_labels = df.Labels.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Cause': 0,
 'Mitigation Efforts': 1,
 'Relief Measures': 2,
 'Regions': 3,
 'Safety Instructions': 4,
 'Emergency contact': 5,
 'News': 6}

In [6]:
df['Labels'] = df.Labels.replace(label_dict)
df

,Query,Labels
0,What makes the ground shake during an earthquake?,0
1,What causes the earth to tremble and buildings...,0
2,What happens underground to create an earthquake?,0
3,What causes the earth to shake violently durin...,0
4,What are the factors that contribute to the oc...,0
...,...,...
305,Has there been any news on the environmental i...,6
306,Are there any reports on the economic conseque...,6
307,Has there been any news on technological advan...,6
308,Are there any reports on community initiatives...,6


In [7]:
!pip install transformers
import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
docs = df.Query.values
labels = df.Labels.values

In [10]:
max_len = 0 #stores the maximum length of a content document

for sentence in docs:
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sentence, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  26


In [11]:
input_ids = []
attention_masks = []

for sentence in docs:
    encoded_dict = tokenizer.encode_plus(
                        sentence,                      # Sentence to be encoded.
                        add_special_tokens = True,     # Add '[CLS]' and '[SEP]'
                        max_length = 58,               # Pad & truncate all sentences
                        pad_to_max_length = True,      # according to maxLength.
                        return_attention_mask = True,  # Construct attention masks.
                        return_tensors = 'pt',         # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And the attention mask
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

#Example conversion
print('Original: ', docs[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  What makes the ground shake during an earthquake?
Token IDs: tensor([ 101, 2054, 3084, 1996, 2598, 6073, 2076, 2019, 8372, 1029,  102,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0])


In [12]:
from torch.utils.data import TensorDataset, random_split

# Training inputs can be combined into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Since we have very limited data, we split the training and
# validation sets in a 9 : 1 ratio, so that we have enough datapoints
# to train/fine tune our model on.

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{} training samples'.format(train_size)) #ensuring right alignment with :>
print('{} validation samples'.format(val_size))

248 training samples
62 validation samples


In [13]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Supply the batch size to the dataloader
batch_size = 4

# Create the DataLoaders for our training and validation sets.
#For training, batches can be randomly selected
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )

# For validation the order doesn't matter, so we can read them sequentially.
validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [14]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",          # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 7,               # the number of labels
    output_attentions = False,    # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model = model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [15]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5,
                  eps = 1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs.
epochs = 8

# Total number of training steps = [number of batches] x [number of epochs].
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [17]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat)/len(labels_flat)

In [18]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [20]:
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training-->')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()
    train_acc=0
    for step, batch in enumerate(train_dataloader):
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Clear any previously calculated gradients before performing a
        # backward pass.
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end.
        total_train_loss += loss.item()
        train_acc += flat_accuracy(logits, label_ids)

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    train_acc /= len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("Training Accuracy : {0:.2f}".format(train_acc))
    print("Average training loss: {0:.2f}".format(avg_train_loss))
    print("Training epoch took: {:}".format(training_time))


    print("")
    print("Running Validation-->")

    t0 = time.time()

    model.eval()

    # Initialising variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            output_val = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)

        loss = output_val[0]
        logits = output_val[1]

        # Keep adding the validation loss.
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("Average Validation Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("Average Validation Loss: {0:.2f}".format(avg_val_loss))
    print("Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))


======== Epoch 1 / 8 ========
Training-->

Training Accuracy : 0.39
Average training loss: 1.69
Training epoch took: 0:00:05

Running Validation-->
Average Validation Accuracy: 0.88
Average Validation Loss: 0.89
Validation took: 0:00:00

======== Epoch 2 / 8 ========
Training-->

Training Accuracy : 0.95
Average training loss: 0.39
Training epoch took: 0:00:06

Running Validation-->
Average Validation Accuracy: 1.00
Average Validation Loss: 0.03
Validation took: 0:00:00

======== Epoch 3 / 8 ========
Training-->

Training Accuracy : 1.00
Average training loss: 0.02
Training epoch took: 0:00:07

Running Validation-->
Average Validation Accuracy: 1.00
Average Validation Loss: 0.01
Validation took: 0:00:00

======== Epoch 4 / 8 ========
Training-->

Training Accuracy : 1.00
Average training loss: 0.01
Training epoch took: 0:00:08

Running Validation-->
Average Validation Accuracy: 1.00
Average Validation Loss: 0.00
Validation took: 0:00:00

======== Epoch 5 / 8 ========
Training-->

Trai

In [21]:
ansDict = {
    'Cause': [
    "Earthquake: Caused by the sudden release of energy in the Earth's crust, usually due to tectonic plate movements. This can occur along fault lines or boundaries where the plates collide, slide past each other, or separate.",
    "Tsunami: Generated by underwater disturbances such as earthquakes, volcanic eruptions, or landslides. These disturbances displace large volumes of water, creating powerful ocean waves that can travel long distances.",
    "Volcanic Eruption: Triggered by the rise of molten rock (magma) from deep within the Earth's mantle. When the pressure becomes too great, the magma is expelled through a volcano, releasing gases, ash, and lava onto the surface.",
    "Hurricane: Formed over warm ocean waters when a combination of low atmospheric pressure, high humidity, and wind patterns create a spinning storm system. The warm ocean water provides the energy source for the hurricane to intensify.",
    "Tornado: Created by the collision of warm, moist air from the south and cold, dry air from the north. Atmospheric instability, temperature contrasts, and wind shear cause the air to rotate vertically, forming a destructive funnel-shaped cloud.",
    "Flood: Caused by excessive rainfall, rapid snowmelt, or the failure of dams or levees. When the amount of water exceeds the capacity of rivers, lakes, or drainage systems, it overflows and inundates surrounding areas.",
    "Drought: Occurs when there is an extended period of significantly below-average precipitation leading to a lack of water supply. High temperatures and evaporation rates exacerbate the drying of soil and vegetation.",
    "Wildfire: Ignited by a combination of dry and hot weather, lightning strikes, or human activities such as campfires or careless burning. Once ignited, wildfires spread rapidly, fueled by dry vegetation and driven by wind.",
    "Blizzard: Characterized by strong winds, blowing snow, and freezing temperatures. Blizzards typically occur during winter storms when there is abundant moisture in the air and a strong low-pressure system.",
    "Landslide: Caused by heavy rainfall saturating the ground, earthquake activity destabilizing slopes, or human activities altering the natural landscape. The gravitational force pulls loose soil and rock downslope, often triggered by a specific event.",
    "Avalanche: Occurs when a mass of snow rapidly slides down a mountainside. Factors that contribute to avalanches include heavy snowfall, steep slopes, changes in temperature, or human activity like skiing or snowmobiling.",
    "Cyclone: Similar to hurricanes, cyclones are large-scale rotating storm systems. They form over warm ocean waters and are characterized by low atmospheric pressure, high humidity, and distinct wind patterns.",
    "Heatwave: A prolonged period of excessively hot weather caused by high-pressure systems that trap warm air in a region. Heatwaves are often intensified by human-induced climate change.",
    "Hailstorm: Generated by updrafts in thunderstorms, where raindrops are lifted to higher altitudes within the storm cloud and freeze into ice pellets. Hailstones grow in size as they are repeatedly carried by updrafts and downdrafts.",
    "Dust Storm: Occurs when strong winds lift and carry large amounts of dry soil or sand particles. Dust storms often happen in arid or semi-arid regions where loose soil and dry weather conditions are prevalent.",
    "Sinkhole: Formed when the ground collapses due to the dissolution of underground limestone or other soluble rocks. This can be a natural process over long periods or can be triggered by human activities like mining or excessive groundwater pumping.",
    "Famine: Caused by a combination of factors, including crop failure due to drought, floods, pests, or diseases, as well as socioeconomic issues, conflict, and lack of access to food and resources.",
    "Epidemic: Refers to the rapid spread of infectious diseases among a large number of people in a specific population or region. Epidemics can be caused by viruses, bacteria, or other pathogens transmitted through various means.",
    "Frost: Occurs when temperatures drop below freezing point, causing the water vapor in the air to condense and freeze on surfaces. Frost can damage crops, plants, and exposed materials.",
    "Mudslide: Triggered by heavy rainfall saturating slopes with loose soil or rock. The excess water increases the weight and reduces the friction between particles, causing them to slide downhill.",
    "Ice Storm: Results from freezing rain, which occurs when raindrops fall through a layer of subfreezing air near the surface. The raindrops freeze upon contact with cold surfaces, including trees, power lines, and roads.",
    "Solar Flare: Eruptions on the sun's surface that release a massive amount of energy, including electromagnetic radiation, high-energy particles, and solar plasma. When directed towards Earth, solar flares can disrupt communication systems and affect the planet's magnetic field.",
    "Tidal Wave: Created by strong ocean currents or undersea volcanic eruptions. Tidal waves, also known as tidal surges or storm surges, are characterized by rapidly rising sea levels and can cause significant coastal flooding.",
    "Heat Exhaustion: Occurs when the body overheats due to prolonged exposure to high temperatures and inadequate fluid intake. It is often associated with physical exertion and can lead to dehydration and other health complications.",
],
 'Regions': [
    "Earthquake: Pacific Ring of Fire (including countries like Japan, Indonesia, and Chile)",
    "Tsunami: Pacific Ring of Fire (including countries like Japan, Indonesia, and Chile)",
    "Volcanic Eruption: Pacific Ring of Fire (including countries like Japan, Indonesia, and Chile)",
    "Hurricane: Atlantic Ocean (particularly the Caribbean Sea and the Gulf of Mexico)",
    "Tornado: Tornado Alley in the United States (central part, including states like Texas, Oklahoma, and Kansas)",
    "Flood: Low-lying areas near major rivers or coastal regions (varies depending on the country and climate)",
    "Drought: Arid and semi-arid regions around the world (such as parts of Africa, Australia, and southwestern United States)",
    "Wildfire: Forested areas prone to dry and hot conditions (including regions in California, Australia, and Mediterranean countries)",
    "Blizzard: Northern regions with cold climates (such as northern United States, Canada, and northern Europe)",
    "Landslide: Hilly or mountainous regions with steep slopes (varies depending on the geography)",
    "Avalanche: Mountainous regions with heavy snowfall (such as the Alps, Rocky Mountains, and Himalayas)",
    "Cyclone: Coastal regions of the Indian Ocean (including countries like India, Bangladesh, and Myanmar)",
    "Heatwave: Regions with hot climates, including deserts and subtropical areas",
    "Hailstorm: Regions with thunderstorm activity and suitable conditions for hail formation (varies globally)",
    "Dust Storm: Arid or semi-arid regions with loose soil or desert areas (such as the Sahara Desert and parts of the United States)",
    "Sinkhole: Regions with limestone or soluble rock formations (such as the Yucatan Peninsula, Florida, and parts of China)",
    "Famine: Areas with high vulnerability to food insecurity and poverty (varies globally)",
    "Epidemic: Can occur globally and depends on the spread of diseases and population density (varies for different outbreaks)",
    "Frost: Regions with cold climates and suitable conditions for frost formation (varies depending on the season)",
    "Mudslide: Hilly or mountainous regions with heavy rainfall or unstable slopes (such as parts of California, Central America, and Southeast Asia)",
    "Ice Storm: Regions with cold climates and precipitation in the form of freezing rain (varies depending on the season)",
    "Solar Flare: Can affect the Earth globally, with potential impacts on satellite communications and power grids",
    "Tidal Wave: Coastal regions near major oceanic fault lines or areas prone to undersea volcanic activity (varies globally)",
    "Heat Exhaustion: Regions experiencing prolonged heatwaves or high temperatures (varies depending on the climate)",
],
'Mitigation Efforts': [
    "Earthquake: Building resilient structures, adhering to building codes, and reinforcing existing infrastructure. Educating communities on emergency preparedness and conducting regular drills.",
    "Tsunami: Developing early warning systems, establishing evacuation plans and routes, and educating coastal communities on tsunami awareness. Constructing tsunami-resistant buildings and structures in vulnerable areas.",
    "Volcanic Eruption: Monitoring volcanic activity and issuing timely warnings. Establishing exclusion zones and evacuation plans for at-risk areas. Educating communities on volcanic hazards and preparedness measures.",
    "Hurricane: Developing hurricane tracking and forecasting systems. Implementing building codes for hurricane-resistant construction. Conducting public awareness campaigns and ensuring evacuation plans are in place.",
    "Tornado: Establishing tornado warning systems and monitoring weather conditions. Constructing tornado-resistant buildings and safe rooms. Educating communities on tornado safety and preparedness.",
    "Flood: Implementing floodplain management measures, such as zoning and land-use regulations. Constructing flood protection infrastructure like levees and flood barriers. Promoting early warning systems and evacuation plans.",
    "Drought: Implementing water conservation and management strategies. Developing drought-resistant agricultural practices. Promoting public awareness about water conservation and implementing water-use restrictions.",
    "Wildfire: Creating defensible spaces around structures, clearing vegetation, and implementing fire-resistant building codes. Conducting controlled burns and managing forest fuel loads. Promoting fire safety education and awareness.",
    "Blizzard: Preparing emergency kits and supplies for severe winter weather. Clearing snow from roads and public areas promptly. Educating communities on winter weather safety and providing heating assistance to vulnerable populations.",
    "Landslide: Implementing land-use planning and slope stabilization measures. Conducting geological assessments and mapping landslide-prone areas. Developing early warning systems and promoting public awareness.",
    "Avalanche: Conducting avalanche forecasting and issuing warnings. Establishing avalanche control measures. Educating communities on backcountry safety and promoting safe recreational practices in avalanche-prone areas.",
    "Cyclone: Developing cyclone forecasting and early warning systems. Constructing cyclone-resistant buildings and shelters. Educating communities on cyclone preparedness and evacuation plans.",
    "Heatwave: Establishing heatwave early warning systems and public cooling centers. Educating communities on heatwave safety and promoting heat illness prevention measures. Providing assistance to vulnerable populations during heatwaves.",
    "Hailstorm: Providing hailstorm-resistant roofing materials and structures. Developing early warning systems for hailstorms. Educating communities on hailstorm safety measures.",
    "Dust Storm: Implementing land and soil conservation practices to reduce soil erosion. Establishing windbreaks and shelterbelts. Educating communities on dust storm safety and promoting proper land management.",
    "Sinkhole: Conducting geological surveys and mapping sinkhole-prone areas. Implementing land-use planning to avoid vulnerable areas. Conducting regular inspections of infrastructure and foundations.",
    "Famine: Implementing sustainable agriculture practices and promoting crop diversification. Developing early warning systems for food security. Providing social safety nets and assistance to vulnerable populations.",
    "Epidemic: Promoting vaccination programs and public health campaigns. Implementing disease surveillance and monitoring systems. Developing emergency response plans and establishing healthcare infrastructure.",
    "Frost: Using frost protection measures for crops, such as irrigation and covering. Educating farmers on frost prevention techniques. Developing early warning systems for frost events.",
    "Mudslide: Implementing slope stabilization measures, such as retaining walls and drainage systems. Conducting land-use planning and avoiding development in landslide-prone areas. Educating communities on mudslide preparedness and evacuation plans.",
    "Ice Storm: Strengthening power infrastructure and conducting regular maintenance. Trimming trees near power lines to prevent ice accumulation. Educating communities on ice storm safety and emergency preparedness.",
    "Solar Flare: Developing robust space weather monitoring systems. Protecting critical infrastructure, such as satellites and power grids, from solar flares. Establishing emergency response plans for space weather events.",
    "Tidal Wave: Developing early warning systems and evacuation plans for coastal communities. Constructing coastal defenses, such as seawalls and dikes. Educating communities on tsunami awareness and safety measures.",
    "Heat Exhaustion: Promoting heat safety education and awareness. Providing access to cool and shaded spaces during heatwaves. Ensuring availability of clean drinking water and encouraging proper hydration.",
    # Earthquake
    "Common mitigation efforts before an earthquake occurs include securing heavy objects and furniture, reinforcing buildings to withstand shaking, and creating an emergency communication plan. It's also important to educate oneself about earthquake safety and participate in drills and preparedness programs.",
    "To prepare for an earthquake, you can take steps such as anchoring tall and heavy furniture, securing loose items, and installing latches on cabinets. Reinforcing the structure of your home, like securing the foundation and reinforcing walls, can also help minimize damage. Creating a communication plan with your family and knowing the location of safe spots within your home are important as well. It is also advisable to have emergency supplies such as food, water, and first aid kits readily available.",
    "Before an earthquake, common mitigation efforts include constructing buildings with earthquake-resistant designs, using flexible materials that can withstand shaking, and reinforcing weak points in structures. Implementing early warning systems can also provide people with crucial seconds to take cover or evacuate.",
    "Efforts to improve early warning systems for earthquakes involve setting up networks of seismic sensors to detect ground shaking and quickly transmit warnings to areas at risk. These warnings can be communicated through mobile apps, sirens, or other alert systems to give people a few seconds or more of advance notice to take protective actions.",

    # Hurricane
    "Common mitigation efforts before a hurricane occurs include creating a family emergency plan, securing or reinforcing windows and doors, and stocking up on essential supplies like food, water, and medications. It's also important to stay informed about the weather conditions and follow evacuation orders if necessary.",
    "To mitigate the impact of a hurricane, you can take steps such as installing storm shutters or using plywood to cover windows and reinforcing garage doors. Clearing your yard of any loose objects that could become projectiles during high winds is important too. Stocking up on non-perishable food, water, and essential supplies is advisable. It's also crucial to stay informed about the hurricane's path and follow evacuation orders issued by local authorities.",
    "Before a hurricane, common mitigation efforts include building codes that require hurricane-resistant construction techniques, such as reinforced roofs and impact-resistant windows. Coastal areas may implement beach nourishment projects or construct seawalls to reduce the impact of storm surge. Emergency management agencies also play a crucial role in providing timely information and coordinating evacuations.",
    "Mitigation efforts to strengthen infrastructure against hurricanes include constructing buildings that can withstand high winds and storm surge, elevating critical infrastructure in flood-prone areas, and implementing strict building codes and regulations. Additionally, maintaining and reinforcing levees, flood control channels, and drainage systems can help minimize the risk of flooding during hurricanes.",

    # Flood
    "Common mitigation efforts before a flood occurs include elevating electrical systems and appliances, installing flood-resistant barriers or floodgates, and creating proper drainage systems. It's important to stay informed about flood risks in your area and have a family emergency plan in place.",
    "To prevent or minimize flood damage, it is advisable to keep gutters and drains clear of debris to ensure proper water flow. Installing a sump pump and a battery-operated backup system can help remove excess water. Raising electrical outlets, switches, and appliances above potential flood levels is also recommended. Additionally, communities can implement flood control measures such as constructing levees, flood walls, and detention basins to mitigate the risk of flooding.",
    "Improving flood forecasting and emergency response systems involves enhancing weather monitoring and river gauging capabilities to detect and predict flooding. Implementing effective communication systems, such as text alerts or sirens, can help quickly notify residents of impending floods. Communities can also develop floodplain management plans and zoning regulations to prevent new development in flood-prone areas.",
    "Efforts to reduce vulnerability to floods include implementing measures like land-use planning and zoning regulations that prohibit or restrict development in flood-prone areas. Additionally, constructing flood-control structures such as levees, flood walls, and detention basins can help redirect and manage floodwaters. Educating the public about flood risks and promoting insurance coverage for flood damage is also important.",

    # Tornado
    "Common mitigation efforts before a tornado occurs include identifying safe rooms or shelters in your home, securing outdoor objects, and staying alert to weather warnings. It's crucial to have a family emergency plan and practice tornado drills.",
    "To stay safe during a tornado, it is important to identify the safest areas in your home, such as a basement or an interior room on the lowest floor. Reinforcing these areas with additional structural support can provide added protection. Securing outdoor objects or bringing them inside can help prevent them from becoming hazards in high winds. Staying informed about tornado warnings and having a battery-powered weather radio or mobile app can provide timely information.",
    "Improving tornado warning systems and preparedness plans involves implementing advanced technologies, such as Doppler radar and storm tracking algorithms, to detect and forecast tornadoes more accurately. Local emergency management agencies play a crucial role in disseminating warnings and conducting public awareness campaigns to ensure residents are prepared for tornadoes. Educating the public about tornado safety and conducting regular tornado drills are also important mitigation efforts.",
    "Mitigation efforts to make buildings more tornado-resistant include implementing stricter building codes and standards for wind resistance. Constructing tornado-safe rooms or shelters within homes, schools, and other public buildings is also important. Additionally, ensuring the structural integrity of roofs, windows, and doors can help prevent or reduce damage caused by tornadoes.",

    # Tsunami
    "Common mitigation efforts before a tsunami occurs include educating residents about evacuation routes and safe zones, establishing early warning systems, and implementing land-use planning that avoids building in tsunami-prone areas. It's important to be aware of tsunami signs and alerts.",
    "To reduce the impact of a tsunami, it is important to know the evacuation routes and safe zones in your area. Familiarize yourself with the signs of a tsunami, such as strong and unusual ocean tides or the sound of approaching waves. Implementing early warning systems, such as sirens or text alerts, can provide valuable time for evacuation. Communities can also establish land-use planning policies that restrict development in high-risk coastal areas.",
    "Before a tsunami occurs, efforts can be made to establish early warning systems that use seismographs and ocean buoys to detect earthquake activity and monitor sea level changes. These systems can provide timely alerts to coastal communities, allowing them to evacuate to safe zones. Public education campaigns and community drills are also important to ensure residents know how to respond when a tsunami warning is issued.",
    "Mitigation efforts to educate communities about evacuation routes and safe zones involve conducting outreach programs and distributing informational materials that explain the signs of a tsunami and provide instructions for evacuation. Implementing effective communication systems, such as sirens or text alerts, can help notify residents in a timely manner. Establishing designated safe zones and clear evacuation routes also plays a crucial role in mitigating the impact of tsunamis.",

    # Volcanic Eruption
    "Common mitigation efforts before a volcanic eruption occurs include establishing evacuation plans and safe zones, monitoring volcanic activity, and educating residents about volcano hazards. It's important to stay informed about the latest volcanic alerts and advisories.",
    "To protect communities from volcanic eruptions, it is important to establish evacuation plans that outline safe routes and designated shelters. Monitoring volcanic activity through the use of seismometers, gas sensors, and satellite imagery can provide early warning signs of an impending eruption. Educating residents about volcano hazards, such as pyroclastic flows and ashfall, can help them understand the risks and take appropriate actions.",
    "Before a volcanic eruption occurs, efforts can be made to improve volcanic monitoring systems, such as installing additional seismometers and gas sensors near active volcanoes. Implementing real-time data analysis and communication networks can help provide timely updates on volcanic activity. Communities can also establish emergency management plans and conduct drills to ensure residents are prepared for volcanic hazards.",
    "Mitigation efforts to establish evacuation plans and safe zones involve working closely with local authorities and emergency management agencies to identify areas at risk and develop evacuation routes and shelters. Educating residents about the hazards associated with volcanic eruptions and providing clear instructions on evacuation procedures are crucial. Implementing effective communication systems, such as sirens or mobile alerts, can help notify residents of volcanic alerts and advisories.",

    # Drought
    "Common mitigation efforts before a drought occurs include implementing water conservation measures, diversifying water sources, and promoting efficient irrigation practices. It's important to use water wisely and be mindful of water consumption.",
    "To cope with water scarcity during a drought, it is important to implement water conservation measures such as fixing leaks, using water-efficient appliances, and reducing outdoor water usage. Diversifying water sources by investing in alternative sources like rainwater harvesting or wastewater recycling can also help. Promoting efficient irrigation practices in agriculture and landscaping can further conserve water.",
    "Efforts to adapt and conserve water during periods of drought involve implementing water-saving technologies and practices, such as drip irrigation or smart irrigation systems, to reduce water usage in agriculture and landscaping. Communities can also promote public awareness campaigns and education programs to encourage water conservation behaviors, such as shorter showers and fixing leaks promptly.",
    "Mitigation efforts to improve water management and storage systems involve implementing water-efficient technologies and infrastructure, such as water recycling and desalination plants. Developing drought contingency plans and water conservation policies can help ensure sustainable water usage. Additionally, promoting public awareness about water conservation and incentivizing water-saving behaviors can contribute to mitigating the impact of drought.",

    # Wildfire
    "Common mitigation efforts before a wildfire occurs include creating defensible spaces around homes, clearing vegetation near structures, and maintaining an emergency supply kit. It's important to be vigilant and report any signs of wildfire immediately.",
    "To prevent and control wildfires, it is important to create defensible spaces by removing flammable vegetation and maintaining a safe distance between structures and trees or shrubs. Clearing debris from gutters, roofs, and yards can also help prevent ignition from embers. Creating a family emergency supply kit with essential items like water, food, and first aid supplies is advisable. Staying informed about wildfire risks and reporting any signs of fire promptly is crucial.",
    "Before a wildfire occurs, communities can create defensible spaces by clearing flammable vegetation, implementing regulations for building materials and landscaping in fire-prone areas, and establishing firebreaks. Conducting controlled burns or prescribed fires under controlled conditions can also help reduce the accumulation of vegetation that can fuel wildfires. Educating the public about fire safety, such as proper campfire and cigarette disposal, is important too.",
    "Mitigation efforts to raise awareness about fire safety and responsible behavior involve implementing public education campaigns that emphasize the importance of following fire safety guidelines. Communities can also establish regulations and ordinances related to fire prevention and implement stricter enforcement measures. Additionally, fostering a culture of community responsibility and encouraging residents to report any suspicious or unsafe activities can help prevent wildfires.",

    # Avalanche
    "Common mitigation efforts before an avalanche occurs include avoiding avalanche-prone areas, carrying avalanche safety equipment, and receiving proper avalanche training. It's important to stay informed about avalanche conditions and heed warnings from authorities.",
    "To reduce the risk of avalanches, it is important to avoid traveling in avalanche-prone areas, especially during periods of high avalanche danger. Carrying avalanche safety equipment such as beacons, shovels, and probes can increase your chances of survival if caught in an avalanche. Seeking proper avalanche training and education is crucial for understanding avalanche terrain and making informed decisions. Staying informed about avalanche conditions through avalanche bulletins and local authorities is also important.",
    "Before an avalanche occurs, efforts can be made to assess and mitigate avalanche hazards in high-risk areas. Implementing avalanche forecasting and warning systems can help provide timely information to backcountry travelers. Establishing guidelines and regulations for safe travel in avalanche terrain and promoting avalanche education and training programs are important mitigation efforts.",
    "Mitigation efforts to improve avalanche safety involve implementing avalanche control measures, such as snowpack stabilization techniques and the use of explosives to trigger controlled avalanches. Conducting regular avalanche forecasting and monitoring snowpack conditions can help provide accurate and timely avalanche warnings. Educating the public about avalanche safety and promoting responsible backcountry travel are crucial aspects of mitigating avalanche risks.",

    # Landslide
    "Common mitigation efforts before a landslide occurs include avoiding landslide-prone areas, stabilizing slopes, and monitoring changes in the landscape. It's important to stay informed about landslide risks and heed evacuation orders.",
    "To minimize the risk of landslides, it is important to avoid building or living in landslide-prone areas. Stabilizing slopes through techniques like retaining walls, terracing, and drainage systems can help prevent landslides. Monitoring changes in the landscape, such as cracks or movement in the ground, can provide early warning signs. Staying informed about landslide risks through local authorities and heeding evacuation orders when issued is crucial.",
    "Before a landslide occurs, efforts can be made to assess and map landslide-prone areas, implement land-use planning that avoids development in high-risk zones, and establish early warning systems. Conducting geotechnical studies to identify unstable slopes and implementing slope stabilization measures can help reduce the likelihood of landslides. Educating the public about landslide hazards and promoting responsible construction practices are important mitigation efforts.",
    "Mitigation efforts to improve slope stability and prevent landslides involve implementing measures such as slope reinforcement techniques, drainage systems, and vegetation management. Conducting geotechnical investigations and engineering studies can help identify areas at risk and design appropriate stabilization measures. Establishing effective communication systems to issue landslide warnings and coordinating emergency response plans are crucial aspects of mitigating landslide risks.",

    # Hailstorm
    "Common mitigation efforts before a hailstorm occurs include having protective covers for vehicles and property, monitoring weather forecasts, and taking shelter during severe storms. It's important to stay informed and be prepared for sudden hailstorms.",
    "To mitigate the impact of a hailstorm, it is advisable to park vehicles in covered areas or use hail blankets or car covers to protect them from hail damage. Protecting windows and fragile objects with shutters or blankets can help safeguard property. Monitoring weather forecasts and heeding warnings from meteorological authorities can provide advanced notice. When a severe hailstorm is approaching, seeking shelter indoors or in a sturdy structure is crucial.",
    "Before a hailstorm occurs, efforts can be made to improve weather forecasting and prediction capabilities to provide timely warnings. Developing hail-resistant building practices, such as using impact-resistant roofing materials and reinforced windows, can help minimize damage. Implementing public awareness campaigns about hailstorm risks and safety measures can also contribute to mitigation.",
    "Efforts to improve weather forecasting and hail prediction involve the use of advanced radar technologies to track storm systems and identify hail-producing cells. Implementing automated warning systems that issue alerts based on real-time data can help provide timely notifications to the public. Promoting hail-resistant building practices and educating homeowners about protective measures, such as installing impact-resistant roofs or shutters, are important aspects of mitigating hailstorm damage."


],
'Relief Measures': [
    "Earthquake: Search and rescue operations, providing medical aid and emergency healthcare services. Temporary shelter and housing assistance for displaced individuals. Infrastructure restoration and rebuilding efforts.",
    "Tsunami: Emergency evacuation and search and rescue operations. Medical aid for the injured. Providing temporary shelter and basic necessities. Rebuilding coastal communities and infrastructure.",
    "Volcanic Eruption: Evacuation of affected areas. Providing medical assistance for respiratory issues and other health concerns. Offering temporary shelter and aid for displaced individuals. Cleaning and restoring affected areas.",
    "Hurricane: Search and rescue operations for stranded individuals. Providing medical assistance and emergency healthcare services. Temporary shelter, food, and clean water distribution. Infrastructure repair and restoration.",
    "Tornado: Search and rescue operations for trapped individuals. Medical aid for the injured. Temporary shelter and assistance for displaced individuals. Cleaning and rebuilding affected areas.",
    "Flood: Evacuation of affected individuals to safe locations. Providing medical aid and emergency healthcare services. Temporary shelter and basic necessities. Conducting damage assessments and initiating flood recovery efforts.",
    "Drought: Providing access to clean water and implementing water conservation measures. Offering assistance and support for affected agricultural communities. Providing food aid and livelihood support.",
    "Wildfire: Evacuation of at-risk areas. Search and rescue operations for stranded individuals. Medical aid for burn injuries and respiratory issues. Providing temporary shelter, food, and essentials. Initiating reforestation and rehabilitation efforts.",
    "Blizzard: Ensuring access to heating and warm shelters. Providing medical aid for cold-related illnesses and injuries. Clearing snow from roads and restoring essential services. Assisting vulnerable populations.",
    "Landslide: Evacuation of affected areas. Conducting search and rescue operations for trapped individuals. Providing medical aid and emergency healthcare services. Rebuilding and stabilizing affected slopes.",
    "Avalanche: Conducting search and rescue operations for buried individuals. Providing medical aid for avalanche-related injuries. Ensuring avalanche safety education and training. Restoring access routes and preventing future avalanches.",
    "Cyclone: Search and rescue operations in affected areas. Providing medical aid and emergency healthcare services. Temporary shelter and assistance for displaced individuals. Rebuilding damaged infrastructure and communities.",
    "Heatwave: Setting up cooling centers and providing access to hydration and shade. Offering medical aid for heat-related illnesses. Ensuring support for vulnerable populations. Raising public awareness on heatwave safety.",
    "Hailstorm: Assessing damage and providing necessary repairs. Offering financial assistance for affected individuals and businesses. Providing temporary shelter and basic necessities. Restoring affected infrastructure.",
    "Dust Storm: Ensuring access to clean air and respiratory protection. Clearing debris from roads and infrastructure. Conducting damage assessments and initiating cleanup and restoration efforts.",
    "Sinkhole: Assessing structural damage and ensuring safety in affected areas. Providing assistance for affected individuals and businesses. Conducting geotechnical investigations and stabilizing the affected land.",
    "Famine: Providing emergency food aid and nutritional support. Implementing agricultural assistance and recovery programs. Offering healthcare services and support to affected populations. Strengthening long-term food security measures.",
    "Epidemic: Implementing disease surveillance and control measures. Providing medical aid and healthcare services. Ensuring access to vaccines and treatments. Educating communities on preventive measures.",
    "Frost: Assessing damage to crops and providing agricultural assistance. Restoring affected infrastructure and services. Offering financial support for affected individuals and businesses.",
    "Mudslide: Search and rescue operations in affected areas. Providing medical aid and emergency healthcare services. Temporary shelter and assistance for displaced individuals. Clearing debris and stabilizing slopes.",
    "Ice Storm: Restoring power and essential services. Providing assistance for individuals affected by power outages and hazardous conditions. Clearing ice from roads and infrastructure. Offering support to vulnerable populations.",
    "Solar Flare: Monitoring and restoring satellite communications and power grids. Implementing contingency plans for future solar flares. Ensuring backup systems and redundancy measures for critical infrastructure.",
    "Tidal Wave: Search and rescue operations in affected coastal areas. Providing medical aid and emergency healthcare services. Offering temporary shelter and aid for displaced individuals. Rebuilding coastal communities and infrastructure.",
    "Heat Exhaustion: Providing access to cooling centers and hydration. Offering medical aid for heat-related illnesses. Raising public awareness on heat safety measures and providing support to vulnerable populations.",
],
'Safety Instructions': [
    "Earthquake: Stay calm and take cover under sturdy furniture or against an interior wall. Protect your head and neck. After the shaking stops, evacuate to a safe area and follow emergency protocols.",
    "Tsunami: Immediately move to higher ground or an elevated area away from the coast. Follow evacuation orders and listen to local authorities for updates. Do not return to coastal areas until given the all-clear.",
    "Volcanic Eruption: Stay informed about volcanic activity through official channels. Follow evacuation orders and relocate to a safe location. Avoid areas downwind of the volcano. Protect yourself from ashfall and airborne pollutants.",
    "Hurricane: Secure your workplace and follow emergency plans. Stay indoors, away from windows, during the storm. Evacuate if instructed to do so. After the hurricane, assess damage cautiously and prioritize safety.",
    "Tornado: Seek shelter in a windowless interior room on the lowest level. Protect your head and neck. If outside, find a low-lying area and lie flat. Avoid mobile homes and vehicles. Follow post-tornado safety protocols.",
    "Flood: Evacuate to higher ground if instructed to do so. Avoid floodwaters and submerged areas. Follow emergency communications and use caution when returning to affected areas. Report any hazards or damages.",
    "Drought: Conserve water and follow local water use restrictions. Implement drought-resistant practices in operations. Seek alternative water sources and use water efficiently.",
    "Wildfire: Follow evacuation orders and move to a safe location. Stay informed about the fire's progress. Protect yourself from smoke and embers. After the wildfire, exercise caution when returning and be aware of residual risks.",
    "Blizzard: Stay indoors and avoid unnecessary travel. Keep emergency supplies and warm clothing accessible. If stranded, stay in the vehicle and call for help. Clear snow safely and be cautious of frostbite and hypothermia.",
    "Landslide: Be alert to signs of slope instability. If evacuation is recommended, leave the area immediately. Avoid areas near steep slopes during heavy rainfall. Report any signs of potential landslides to authorities.",
    "Avalanche: Educate yourself on avalanche safety and travel in avalanche-prone areas with caution. Carry appropriate safety equipment and know how to use it. Avoid steep slopes during times of heightened avalanche risk.",
    "Cyclone: Stay informed through official sources and follow evacuation orders. Secure your workplace and protect windows. Stay indoors during the storm. After the cyclone, be cautious of hazards and follow safety guidelines.",
    "Heatwave: Stay hydrated and avoid prolonged exposure to heat. Seek shade or air-conditioned spaces when possible. Follow heat safety protocols, especially for outdoor work. Check on colleagues and report any heat-related concerns.",
    "Hailstorm: Seek shelter indoors and protect yourself from hail impacts. Stay away from windows and glass. After the storm, assess any damages and follow safety procedures for repairs.",
    "Dust Storm: Stay indoors and avoid unnecessary travel. Close windows and doors to prevent dust entry. Use air filters or masks if necessary. Resume outdoor activities when visibility improves.",
    "Sinkhole: Be aware of signs of ground subsidence and report any abnormalities. If a sinkhole occurs, evacuate the area and contact authorities. Follow safety protocols for assessment and repairs.",
    "Famine: Follow food security guidelines and support sustainable agricultural practices. Conserve food resources and report any food scarcity concerns. Seek assistance from local authorities or relief organizations.",
    "Epidemic: Practice good hygiene, such as frequent handwashing and covering coughs and sneezes. Follow health advisories and get vaccinated if available. Report any suspected cases or symptoms.",
    "Frost: Protect sensitive equipment and crops from frost damage. Implement frost prevention measures and monitor weather forecasts. Follow recommended practices for frost protection.",
    "Mudslide: Be vigilant for signs of slope instability and heed evacuation warnings. Move to higher ground if necessary. Avoid valleys and areas susceptible to mudslides during heavy rainfall.",
    "Ice Storm: Exercise caution when walking or driving on icy surfaces. Use traction devices or salt for pathways. Monitor weather updates and follow safety recommendations during icy conditions.",
    "Solar Flare: Follow guidance from authorities and maintain backup power sources. Protect sensitive equipment from power surges. Stay informed about space weather updates and potential impacts.",
    "Tidal Wave: Move to higher ground or an elevated area away from the coast. Follow evacuation orders and listen to local authorities for updates. Avoid returning to coastal areas until declared safe.",
    "Heat Exhaustion: Stay hydrated and take breaks in cool areas. Avoid strenuous activities during peak heat. Recognize symptoms of heat exhaustion and seek medical attention if needed.",
],
'Emergency contact': [
    "In case of an emergency in the United States, dial 911",
    "If you are in the United Kingdom and need immediate assistance, call 999 or 112",
    "For emergencies in Australia, the contact number is 000",
    "In Canada, the emergency contact number is 911",
    "If you require emergency assistance in India, dial 112 or use the specific numbers: 100 for police, 101 for fire, and 102 for ambulance",
    "For emergencies in Germany, dial 112",
    "In France, you can reach emergency services by dialing 112 or 18 for firefighters and 15 for medical emergencies",
    "If you are in Japan and need immediate help, call 110 for police or 119 for fire and medical emergencies",
    "In China, dial 110 for police emergencies or 119 for fire and medical emergencies",
    "In Brazil, the emergency contact numbers are 190 for military police, 192 for ambulance services, and 193 for firefighters",
    "If you are in Mexico and require emergency assistance, dial 911",
    "In South Africa, the emergency contact numbers are 10111 for police, 10177 for ambulance services, and 1022 for firefighters",
    "If you are in Nigeria and need immediate help, dial 112 or 199 for police, 112 or 1123 for ambulance services, and 112 or 1122 for firefighters",
    "In Russia, the emergency contact number is 112",
    "For emergencies in Saudi Arabia, dial 112 or 911",
    "In the United Arab Emirates, the emergency contact number is 999",
    "If you are in Pakistan and require immediate assistance, dial 15 for police, 16 for ambulance services, and 16 for firefighters"
]

}

In [22]:
!pip install sentence_transformers

In [38]:
import requests

def fetch_disaster_news(api_key, keyword = None):
    url = 'https://newsapi.org/v2/everything'
    params = {
        'q': keyword or 'natural disaster',
        'apiKey': api_key,
        'language': 'en',
        'sortBy': 'relevancy',
        'pageSize': 5  # Adjust the number of news articles to fetch as per your requirement
    }

    response = requests.get(url, params=params)
    data = response.json()

    if response.status_code == 200:
        articles = data['articles']
        return [(article['title'], article['publishedAt']) for article in articles]
    else:
        print(f'Error: {data["message"]}')

In [36]:
test_queries = ['Why does the earth shake?', 'What relief efforts are undertaken after a hurricane hits?',
                'What measures to take in advance to protect from an Avalanche?',
                'I am in UK. Whom should I call for help?',
                'What to do if I am stuck in a landslide?',
                'Give latest news about Tornado']

In [24]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [25]:
def getTokenisedQueries(queries):
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  for sentence in queries:
      encoded_dict = tokenizer.encode_plus(
                          sentence,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 75,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True, # Construct attention masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )

      # Add the encoded sentence to the list.
      input_ids.append(encoded_dict['input_ids'])

      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)

  return input_ids, attention_masks

In [26]:
def getPrediction(queries):
  categoryMap = {0:'Cause', 1:'Mitigation Efforts',
  2: 'Relief Measures',
  3: 'Regions',
  4: 'Safety Instructions',
  5: 'Emergency contact',
  6: 'News'}

  model.eval()
  predictions = []
  final_pred = []

  input_ids, attention_masks = getTokenisedQueries(queries)

  # Set the batch size.
  batch_size = 4

  # Create the DataLoader.
  prediction_data = TensorDataset(input_ids, attention_masks)
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
  for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)

    # Unpack inputs from our dataloader
    b_input_ids, b_input_mask = batch

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask)

    logits = outputs[0]
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()

    # Store predictions and true labels
    predictions.append(logits)

  for preds in predictions:
    pred_flat = np.argmax(preds, axis=1).flatten()
    for ele in pred_flat:
      final_pred.append(ele)

  categoryWisePreds = [categoryMap[x] for x in final_pred]
  return dict(zip(queries, categoryWisePreds))

In [27]:
def get_answers(queries, ansDict):
  queryCategories = getPrediction(queries)
  questionAnsMapping = {}
  for query in queries:
    queryEmbedding = embedding_model.encode(query)
    if queryCategories[query] == 'News':
      keyword = 'Natural Disaster'
      query_split = query.split()
      keyword_list = ['Earthquake', 'Flood', 'Tornado', 'Blizzard', 'Hurricane', 'Tsunami', 'Wildfire', 'Drought']
      for word in keyword_list:
        if word in query_split:
          keyword = word
          break
      questionAnsMapping[query] = fetch_disaster_news(api_key, keyword)
      continue
    ansEmbeddings = embedding_model.encode(ansDict[queryCategories[query]])
    simValues = cosine_similarity([queryEmbedding], ansEmbeddings)[0]
    simDict = dict(zip(ansDict[queryCategories[query]], simValues))
    simSorted = dict(sorted(simDict.items(), key=lambda x:x[1], reverse = True))

    questionAnsMapping[query] = list(simSorted.keys())[0]
  return questionAnsMapping

In [37]:
queryAnsDict = get_answers(test_queries, ansDict)
pd.DataFrame({'Queries': queryAnsDict.keys(), 'Answers': queryAnsDict.values()})

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


,Queries,Answers
0,Why does the earth shake?,Earthquake: Caused by the sudden release of en...
1,What relief efforts are undertaken after a hur...,Hurricane: Search and rescue operations for st...
2,What measures to take in advance to protect fr...,"Before an avalanche occurs, efforts can be mad..."
3,I am in UK. Whom should I call for help?,If you are in the United Kingdom and need imme...
4,What to do if I am stuck in a landslide?,Landslide: Be alert to signs of slope instabil...
5,Give latest news about Tornado,"[(Deadly tornado strikes small Texas town, 202..."
